In [ ]:

	format=log_format)



In [ ]:
def mdToBatches(data: str, max_tokens: int = 512, print_messages: bool = False) -> list:
	batches = []
	total_tokens = 0
	truncation_counter = 0  # Counter for truncations

	# Split the data into sections based on H1 headings
	sections = re.split(r"(?m)^#\s+", data)[1:]

	for section in sections:
		# Extract the H1 heading
		h1_match = re.match(r"^(.*?)$", section, re.MULTILINE)
		h1 = h1_match.group(1).strip() if h1_match else ""

		# Split the section into subsections based on H2 headings
		subsections = re.split(r"(?m)^##\s+", section)[1:]

		for subsection in subsections:
			# Extract the H2 heading
			h2_match = re.match(r"^(.*?)$", subsection, re.MULTILINE)
			h2 = h2_match.group(1).strip() if h2_match else ""

			# Extract the text content
			text = re.sub(r"^(#|##).*$", "", subsection, flags=re.MULTILINE).strip()

			# Format the entry as (H1)[H2] "text"
			entry = f"({h1})[{h2}] \"{text}\""

			tokens_description = num_tokens(entry)
			if tokens_description <= max_tokens:
				batches.append(entry)
				total_tokens += tokens_description
			else:
				# Truncate and create new batches with the remaining text
				remaining_text = text
				while len(remaining_text) > 0:
					truncated_text = truncated_string(remaining_text, model="gpt-3.5-turbo", max_tokens=max_tokens)
					truncated_entry = f"({h1})[{h2}] \"{truncated_text}\""
					batches.append(truncated_entry)
					total_tokens += num_tokens(truncated_entry)
					truncation_counter += 1
					remaining_text = remaining_text[len(truncated_text):]

	approximate_cost = 0 #TODO: Update
	average_tokens_per_batch = total_tokens / len(batches)
	
	log_data = {
		"TOTAL NUMBER OF BATCHES": len(batches),
		"TOTAL NUMBER OF TOKENS": total_tokens,
		"MAX TOKENS PER BATCH": max_tokens,
		"NUMBER OF TRUNCATIONS": truncation_counter,
		"AVERAGE NUMBER OF TOKENS PER BATCH": round(average_tokens_per_batch, 2),
		"APPROXIMATE COST OF EMBEDDING": f"${round(approximate_cost, 2)} USD"
	}
	
	logging.info(json.dumps(log_data))

	if print_messages:
		for i, batch in enumerate(batches, start=1):
			print(f"Batch {i}:")
			print(batch)
			print(f"Tokens per batch:", num_tokens(batch))
			print("\n")
		print(log_data)

	return batches

In [ ]:
def formatPDF(pdf_file_path: str) -> str:
	loader = PyMuPDFLoader(pdf_file_path)

	pdf_data = loader.load()

	pdf_data

	data = []

	def clean_pdf(content):
		content = re.sub(r'\s+', ' ', content)
		lines = [line.strip() for line in content.splitlines() if line.strip()]
		cleaned_content = '\n'.join(lines)
		return cleaned_content

	for page in pdf_data:
		_text = page.page_content
		text = clean_pdf(_text)
		print(text)

		data.append(text)
	
	return data

In [ ]:

def pdfToBatches(data: list, max_tokens: int = 512, print_messages: bool = True) -> list:
	batches = []
	total_tokens = 0
	truncation_counter = 0  # Counter for truncations

	for entry in data:
		#text = " ".join(i)  # Join the elements of the list into a single string
		tokens_description = num_tokens(entry)
		if tokens_description <= max_tokens:
			batches.append(entry)
		else:
			#TRUNCATE IF STRING MORE THAN x TOKENS
			job_truncated = truncated_string(entry, model="gpt-3.5-turbo", max_tokens=max_tokens)
			batches.append(job_truncated)
			truncation_counter += 1

		total_tokens += num_tokens(entry)

	approximate_cost = 0

	average_tokens_per_batch = total_tokens / len(batches)
	
	log_data = {
		"TOTAL NUMBER OF BATCHES": len(batches),
		"TOTAL NUMBER OF TOKENS": total_tokens,
		"MAX TOKENS PER BATCH": max_tokens,
		"NUMBER OF TRUNCATIONS": truncation_counter,
		"AVERAGE NUMBER OF TOKENS PER BATCH": round(average_tokens_per_batch, 2),
		"APPROXIMATE COST OF EMBEDDING": f"${round(approximate_cost, 2)} USD"
	}
	
	logging.info(json.dumps(log_data))

	if print_messages:
		for i, batch in enumerate(batches, start=1):
			print(f"Batch {i}:")
			print("".join(batch))
			print(f"Tokens per batch:", num_tokens(batch))
			print("\n")

		print(log_data)
	
	return batches


In [ ]:
#Call either md or pdf
batches = mdToBatches(CohereSafetyGuideline, 512)
#batches = pdfToBatches(data, 512)

In [ ]:
batches

In [ ]:
"""import time

for i in range(0, len(batches), 8):
    time.sleep(2)
    chunk = batches[i:i+10]
    clean_chunk = askAnthropic(cleanEntries, chunk)
    print(clean_chunk)"""

In [8]:


vo = Client(api_key=VOYAGE_API_KEY)

result = vo.embed(batches, model="voyage-2", input_type="document", truncation=False)

pd_data = {
    "id": list(range(len(batches))), 
	"chunks": batches,
	"embeddings": result.embeddings
}

df = pd.DataFrame(pd_data)

print(df)

    id                                             chunks  \
0    0  (🤖 Introduction to Command)[🪸 CoraltheChatbot]...   
1    1  (🎙 WritingEffectivePrompts)[✏ PromptWritingTip...   
2    2  (🎙 WritingEffectivePrompts)[✏ PromptWritingTip...   
3    3  (✅ IdealModelBehavior)[BehaviorPrinciples] "Be...   
4    4  (✅ IdealModelBehavior)[WritingDefaults] "Writi...   
5    5  (✅ IdealModelBehavior)[WritingDefaults] "```\n...   
6    6  (✅ IdealModelBehavior)[Capabilities and Limita...   
7    7  (✅ IdealModelBehavior)[Capabilities and Limita...   
8    8  (✅ IdealModelBehavior)[Capabilities and Limita...   
9    9  (✅ IdealModelBehavior)[Self-Referencevs.Self-A...   
10  10  (✅ IdealModelBehavior)[Self-Referencevs.Self-A...   
11  11  (✅ IdealModelBehavior)[🦺 Safety] "🦺 Safety\n\n...   
12  12  (✅ IdealModelBehavior)[🦺 Safety] "ensual.\n```...   
13  13  (✅ IdealModelBehavior)[🦺 Safety] "self-anthrop...   
14  14  (✅ IdealModelBehavior)[🦺 Safety] " WhatisMeinK...   
15  15  (✅ IdealModelBeh

In [ ]:
#to_postgre(df, table="apricot_moose")